In [ ]:
##### ESTE CODIGO LO HICIMOS CON NICO 16/10/2024. SE USA PARA LOS VIDEOS DE ESCAPE. SU FUNCIÓN ES LA DE OBTENER LAS VARIABLES CINÉTICAS SOLO PARA LOS TRIALS EN DONDE SE CATALOGARON RESPUESTAS, Y ALREDEDOR DE LAS MISMAS (10 O 20 FRAMES) CALCULAR LAS VARIABLES Y OBTENER UNA DISTRIBUCIÓN DE CADA UNA, PARA LOS AISLADOS Y PARA LOS SOCIALES.

import os
import pandas as pd
import matplotlib.pyplot as plt

Tracking=r"C:\Users\Usuario\Desktop\Prueba8\Output"
os.chdir(Tracking)

archivos = os.listdir()
nuevos_archivos = []

for archivo in archivos:
    if '.xlsx' in archivo and 'modified_filtrado_modified_filtrado' in archivo:
        nuevos_archivos.append(archivo)
        
archivos = nuevos_archivos

# Definir la carpeta que contiene los archivos Excel
input_file = r'C:\Users\Usuario\Desktop\Escape nuevo.xlsx'

# Leer el archivo Excel
df = pd.read_excel(input_file)


resultados = {"A": {}, "S": {}}

contador = 0

for Nombre in archivos:
    #if contador > 100: #
    #    break          #
    
    contador = contador + 1
    
    datos = Nombre.split("modified_filtrado_modified_filtrado_")[1].split("DLC")[0].split("_")
    Camada = int(datos[0].split("C")[-1])
    Individuo = int(datos[1])
    Condicion = datos[2]
    Estimulos = datos[3].split("(")[-1].split(")")[0]
    Visual=int(Estimulos.split("-")[0])
    Auditivo=float(Estimulos.split("-")[1])

    df_1 = df[df["Condicion"] == Condicion]
    df_2 = df_1[df_1["Camada"] == Camada]
    df_3 = df_2[df_2["Individuo"] == Individuo]
    df_4 = df_3[df_3["Visual"] == Visual]
    df_5 = df_4[df_4["Auditivo"] == Auditivo]
    
    #print(Camada, Individuo, Condicion, Visual, Auditivo)
    
    Respuesta=df_5["Respuesta"]
    Respuesta=int(Respuesta.iloc[0])
    
    print(Camada, Individuo, Condicion, Visual, Auditivo, Respuesta)
    
    if Respuesta==0:
        continue

    Frame=df_5["Frame"]
    Duracion=df_5["Duracion"]
    
    Frame=int(Frame.iloc[0])
    Duracion=int(Duracion.iloc[0])

    Trackingdf=pd.read_excel(Nombre)

    ventana = 10

    t_inicial = Frame - ventana
    t_final = Frame + ventana

    cant_frames = ventana*2 - 1

    evento = Trackingdf[(Trackingdf["bodyparts_coords"] > t_inicial) & (Trackingdf["bodyparts_coords"] < t_final)]
    
    #print(evento)
    
    if len(evento) != cant_frames:
        continue
    
    print("Evento válido.")
    
    #plt.plot(evento["bodyparts_coords"], evento["head_velocity"])
    #plt.show()
    
    vmax = evento["head_velocity"].max()
    
    if not "velocidad_maxima" in resultados[Condicion]:
        resultados[Condicion]["velocidad_maxima"] = []
    
    resultados[Condicion]["velocidad_maxima"].append(vmax)
    
    angularmax = evento["angular_velocity"].max()
    
    if not "velocidad_angular_maxima" in resultados[Condicion]:
        resultados[Condicion]["velocidad_angular_maxima"] = []
    
    resultados[Condicion]["velocidad_angular_maxima"].append(angularmax)
    
    curvaturemax = evento["sum_curvature"].max()
    
    if not "curvatura_maxima" in resultados[Condicion]:
        resultados[Condicion]["curvatura_maxima"] = []
    
    resultados[Condicion]["curvatura_maxima"].append(curvaturemax)
    
resultados

In [ ]:
#### Esta celda me grafica los datos en base a la condición, y luego en base a las variables que puse arriba. Es decir voy a tener para cada grupo, un grafico con la distribución de cada variable (eje X), por lo tanto voy a tener 3 gráficos (head_velocity, angular_velocity, sum_curvature) para cada grupo.

for condicion in resultados:
    print(condicion)
    
    for variable in resultados[condicion]:
        print(variable)
        
        valores = resultados[condicion][variable]
        
        plt.hist(valores)
        plt.show()